In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

2024-05-19 17:28:03.664374: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 17:28:03.734862: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 17:28:04.053856: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
SECONDS_PER_EPISODE = 25

N_CHANNELS = 3
HEIGHT = 240
WIDTH = 320


SHOW_PREVIEW = True

FIXED_DELTA_SECONDS = 0.2

In [3]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    discrete_actions = {
    0: [-0.5, 1], 1: [0, 1], 2: [0.5, 1], 3: [0, 0]}
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.actor_list = []

        self.action_space = spaces.Discrete(len(self.discrete_actions))

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)

        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)

        self.client.load_world('Town01')

        self.world = self.client.get_world()

        self.settings = self.world.get_settings()
        self.settings.no_rendering_mode = False

        self.world.apply_settings(self.settings)

        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.cleanup()
        
        self.collision_hist = []
        self.actor_list = []
        self.spawn_points = self.world.get_map().get_spawn_points()
        
        self.transform = self.spawn_points[2]
        
        # Define point A and point B
        self.point_a = self.transform.location
        self.point_b = self.spawn_points[241].location
        
        print("Trying to spawn vehicle...")
        self.vehicle = self.world.try_spawn_actor(self.model_3, self.transform)
        
        if self.vehicle is None:
            print("Failed to spawn vehicle! Resetting environment...")
            self.cleanup()
            return self.reset()
        
        print("Vehicle spawned successfully!")
        self.actor_list.append(self.vehicle)
        
        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")

        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z, x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2)  # not detect a collision when the car spawns/falls from sky.

        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))
     
        while self.front_camera is None:
            time.sleep(0.01)  # Just in case car takes any longer.

        self.episode_start = time.time()
        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        # Show camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        
        observation = self.front_camera

        return observation.astype(np.uint8), {}
        
    def step(self, action):
        self.step_counter += 1
        steer, throttle = self.discrete_actions[action]

        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

        self.vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer, brake=0.0))

        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        
        
        # Decelerate as the vehicle approaches the target(Smooth Stopping)
        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        if distance_to_destination < 90.0:  # Within 70 meters of the target
            throttle = max(0.1, throttle * (distance_to_destination / 90.0))  # never below 0.1 3shan mto2afsh 5ales abl ma nwsl
            

        self.vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer, brake=0.0))

    
        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)

        # Initialize variables
        done = False
        terminated = False
        truncated = False
        reward = 0

        # Check for collision
        if len(self.collision_hist) != 0:
            print("Collision occurred")
            reward = -5
            done = True
            terminated = True
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))  # stop the car
            observation = self.reset()[0]
            return observation.astype(np.uint8), reward, terminated, truncated, {}  # Return an empty dictionary as info

        # Check if the car has reached the goal point
        if distance_to_destination < 10.0:  # Threshold for reaching the destination
            print("Goal reached")
            reward = 5
            done = True
            terminated = True
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))  # stop the car
            observation = self.reset()[0]
            return observation.astype(np.uint8), reward, terminated, truncated, {}  # Return an empty dictionary as info

        # Check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            print("Time limit exceeded")
            done = True
            truncated = True
            self.cleanup()
            observation = self.front_camera
            return observation.astype(np.uint8), 0, terminated, truncated, {}  # Return an empty dictionary as info

        # Calculate reward based on distance traveled towards point B
        reward = 1 / distance_to_destination  # Inverse proportional to distance

        # Get observation
        observation = self.front_camera

        return observation.astype(np.uint8), reward, terminated, truncated, {}  # Return an empty dictionary as info

    def cleanup(self):
        for actor in self.actor_list:
            if actor.is_alive:
                actor.destroy()
        cv2.destroyAllWindows()

    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3]  # Ignore the 4th Alpha channel - up to 3
        self.front_camera = i / 255.0

    def collision_data(self, event):
        self.collision_hist.append(event)

In [4]:
'''
env = CarEnv()

check_env(env)
'''

'\nenv = CarEnv()\n\ncheck_env(env)\n'

In [5]:
model_name = "four_throttle_four_steer"
models_dir = f"models/{model_name}/"

logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('CnnPolicy', env, verbose=1,learning_rate=0.001, tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    try:
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print('Iteration', iters, 'has been trained')
        model.save(f"{models_dir}/{TIMESTEPS*iters}")
    except ValueError as e:
        print(f"Error during training iteration {iters}: {e}")
        continue

Trying to spawn vehicle...
Vehicle spawned successfully!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Iteration  1  is to start...
Trying to spawn vehicle...
Vehicle spawned successfully!
Logging to logs/1715836910/PPO_0
-----------------------------
| time/              |      |
|    fps             | 328  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.53e+03    |
|    ep_rew_mean          | 26.6        |
| time/                   |             |
|    fps                  | 183         |
|    iterations           | 2           |
|    time_elapsed         | 22          |
|    tota

Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.05e+03      |
|    ep_rew_mean          | 8.54          |
| time/                   |               |
|    fps                  | 136           |
|    iterations           | 10            |
|    time_elapsed         | 150           |
|    total_timesteps      | 20480         |
| train/                  |               |
|    approx_kl            | 0.00048334876 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.37         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | -0.00723      |
|    n_updates            | 90            |
|    policy_gradient_loss | -0.00036      |
|    value_loss           | 0.0

Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.03e+03      |
|    ep_rew_mean          | 9.58          |
| time/                   |               |
|    fps                  | 130           |
|    iterations           | 18            |
|    time_elapsed         | 282           |
|    total_timesteps      | 36864         |
| train/                  |               |
|    approx_kl            | 0.00046390216 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.36         |
|    explained_variance   | -1.19e-07     |
|    learning_rate        | 0.001         |
|    loss                 | 0.

Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.89e+03     |
|    ep_rew_mean          | 8.62         |
| time/                   |              |
|    fps                  | 126          |
|    iterations           | 26           |
|    time_elapsed         | 421          |
|    total_timesteps      | 53248        |
| train/                  |              |
|    approx_kl            | 0.0014386342 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    loss                 | 0.0116       |
|  

Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.98e+03      |
|    ep_rew_mean          | 10.6          |
| time/                   |               |
|    fps                  | 126           |
|    iterations           | 34            |
|    time_elapsed         | 550           |
|    total_timesteps      | 69632         |
| train/                  |               |
|    approx_kl            | 0.00039633756 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.38         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | -0.00319      |
|    n_updates            | 330           |
|    policy_gradie

...

Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.34e+03    |
|    ep_rew_mean          | 3.25        |
| time/                   |             |
|    fps                  | 131         |
|    iterations           | 213         |
|    time_elapsed         | 3305        |
|    total_timesteps      | 1941504     |
| train/                  |             |
|    approx_kl            | 0.002599413 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.02       |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | 0.104       |
|    n_updates            | 9470        |
|    policy_gradient_loss | -0.00156    |
|    value_loss           | 0.132       |
-----------------------------------------
Collision occurred
Trying to spawn vehicle...
Vehicle spawned

Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.12e+03      |
|    ep_rew_mean          | 1.83          |
| time/                   |               |
|    fps                  | 130           |
|    iterations           | 220           |
|    time_elapsed         | 3448          |
|    total_timesteps      | 1955840       |
| train/                  |               |
|    approx_kl            | 0.00013552947 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.02         |
|    explained_variance   | -1.19e-07     |
|    learning_rate        | 0.001         |
|    loss                 | 0.397         |
|    n_updates            | 9540          |
|    policy_gradient_loss | -6.49e-05     |
|    value_loss           | 0.132         |
-------------------------------------------
Collision occurred


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.08e+03     |
|    ep_rew_mean          | 1.36         |
| time/                   |              |
|    fps                  | 129          |
|    iterations           | 227          |
|    time_elapsed         | 3589         |
|    total_timesteps      | 1970176      |
| train/                  |              |
|    approx_kl            | 0.0017962399 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.984       |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    loss                 | 0.115        |
|    n_updates            | 9610         |
|    policy_gradient_loss | -0.00101     |
|    value_loss           | 0.132        |
------------------------------------------
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehic

Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.08e+03     |
|    ep_rew_mean          | 1.29         |
| time/                   |              |
|    fps                  | 128          |
|    iterations           | 235          |
|    time_elapsed         | 3748         |
|    total_timesteps      | 1986560      |
| train/                  |              |
|    approx_kl            | 0.0002501814 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.992       |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    loss                 | 0.02         |
|  

Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.08e+03    |
|    ep_rew_mean          | 1.28        |
| time/                   |             |
|    fps                  | 127         |
|    iterations           | 242         |
|    time_elapsed         | 3886        |
|    total_timesteps      | 2000896     |
| train/                  |             |
|    approx_kl            | 0.001102658 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.01       |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | 0.188       |
|    n_updates            | 9760        |
|    policy_gradient_loss | -0.000821   |
|    value_loss           | 0.132       |
---------------------------

In [6]:
from stable_baselines3 import results_plotter

ImportError: cannot import name 'results_plotter' from 'stable_baselines3' (/home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/stable_baselines3/__init__.py)

In [6]:
'''
discrete_actions = {
    0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}
Action 0: Steer left (-1) and throttle (1)
Action 1: Do not steer (0) and throttle (1)
Action 2: Steer right (1) and throttle (1)
Action 3: Do not steer (0) and do not throttle (0)
'''

'\ndiscrete_actions = {\n    0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}\nAction 0: Steer left (-1) and throttle (1)\nAction 1: Do not steer (0) and throttle (1)\nAction 2: Steer right (1) and throttle (1)\nAction 3: Do not steer (0) and do not throttle (0)\n'

In [7]:
#discrete_actions = {
 #   0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}

In [6]:
#len(discrete_actions)

In [4]:
env = CarEnv()
env.reset()

models_dir = "/home/gp-coopperc/CARLA_TEST/models/four_throttle_four_steer/"
model_path = f"{models_dir}/2000000.zip"
model = PPO.load(model_path, env=env)

Trying to spawn vehicle...
Vehicle spawned successfully!
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor


env = Monitor(env) 
evaluate_policy(model, env, n_eval_episodes=5,deterministic=True, return_episode_rewards= True)

Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Collision occurred
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!


([-2.750304, -2.772784, -2.666299, -2.716342, -2.784455],
 [402, 398, 417, 408, 396])